<a href="https://colab.research.google.com/github/Nataleca/Diplomado-Natalia-Leon/blob/main/ColabGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Diplomado en IA y AP**
# **Leidy Natalia León Carvajal**
# **Tarea 2: Parte 2**

# Entendiendo Google-Colab: GPU

## Lectura del conjunto de datos:

In [ ]:
import numpy as np
import os
import gzip

def cargar_datos(ruta, tipo='train'):

  ruta_categorias = os.path.join(ruta, '%s-labels-idx1-ubyte.gz' %tipo)
  ruta_imagenes=os.path.join(ruta,'%s-images-idx3-ubyte.gz' %tipo)

  with gzip.open(ruta_categorias,'rb') as rut_cat:
    etiquetas=np.frombuffer(rut_cat.read(),dtype=np.uint8,offset=8)

  with gzip.open(ruta_imagenes,'rb') as rut_imgs:
    imagenes=np.frombuffer(rut_imgs.read(),dtype=np.uint8,offset=16).reshape(len(etiquetas),784)

  return imagenes, etiquetas

## Para acceder a Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

ruta='gdrive/My Drive/IA y AP/fashion_mnist_data'

X_train, Y_train = cargar_datos(ruta, tipo='train')
X_test, Y_test = cargar_datos(ruta, tipo='test')

Mounted at /content/gdrive


## Ajustando el Tamaño de los datos (Múltiplos de 128 para comparar con TPU)

In [ ]:
ntrain=len(X_train)-len(X_train)%128
ntest=len(X_test)-len(X_test)%128
X_train = X_train[0:ntrain,:]
X_test = X_test[0:ntest,:]
Y_train = Y_train[0:ntrain]
Y_test = Y_test[0:ntest]
X_train = np.reshape(X_train,(ntrain,28,28,1))
X_test = np.reshape(X_test,(ntest,28,28,1)) 

## Importando ***Tensorflow 2***

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

print('Versión de Tensorflow: ' + tf.__version__)

Versión de Tensorflow: 2.4.1


### Creación del modelo con ***Keras***

In [ ]:
tf.random.set_seed(200)
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:])) # Normalización
model.add(tf.keras.layers.Conv2D(64, (5,5), padding='same', activation='elu')) # Filtro Convolucional
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2))) # Capa MaxPooling
model.add(tf.keras.layers.Dropout(0.25)) # Capa Dropout

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:])) # Normalización
model.add(tf.keras.layers.Conv2D(128, (5,5), padding='same', activation='elu')) # Filtro Convolucional
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))# Capa MaxPooling
model.add(tf.keras.layers.Dropout(0.25)) # Capa Dropout

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:])) # Normalización
model.add(tf.keras.layers.Conv2D(256, (5,5), padding='same', activation='elu')) # Filtro Convolucional
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))# Capa MaxPooling
model.add(tf.keras.layers.Dropout(0.25)) # Capa Dropout

model.add(tf.keras.layers.Flatten()) # Aplanamos datos
model.add(tf.keras.layers.Dense(256)) # Agregamos red neuronal (256 neuronas)
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10)) # Salida de 10 neuronas
model.add(tf.keras.layers.Activation('softmax')) # Función de activación 'softmax'

# Imprime en pantalla la información del modelo
model.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 28, 28, 1)         4         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 64)        1664      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 128)       204928    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 128)         0

In [ ]:
# Optimizador 'adam'
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

## Entrenando con **GPU**

Verificamos la disponibilidad de la **GPU**:

In [ ]:
nombre_gpu = tf.test.gpu_device_name()
if nombre_gpu != '/device:GPU:0':
  raise SystemError('GPU no encontrada')
print('GPU encontrada: {}'.format(nombre_gpu))

GPU encontrada: /device:GPU:0


Se realizan **2 iteraciones**

In [ ]:
import timeit

def train_gpu():
  with tf.device('/GPU:0'):
    model.fit(X_train,Y_train,validation_data=(X_test,Y_test),batch_size=128,epochs=2,verbose=1)
  
  return None

gpu_time = timeit.timeit('train_gpu()', number=1, setup='from __main__ import train_gpu')

Epoch 1/2
468/468 [==============================] - 8s 17ms/step - loss: 0.5943 - accuracy: 0.8037 - val_loss: 0.3720 - val_accuracy: 0.8684
Epoch 2/2
468/468 [==============================] - 8s 17ms/step - loss: 0.3865 - accuracy: 0.8654 - val_loss: 0.3447 - val_accuracy: 0.8805


In [ ]:
print('Tiempo de entrenamiento: ' + str(gpu_time) + ' segundos!!!')

Tiempo de entrenamiento: 15.811976633000029 segundos!!!


Tiempo de entrenamiento: 15.811976633000029 segundos!!!